In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-price-new/HousePricePrediction.xlsx


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error, mean_absolute_error

In [2]:
data = pd.read_excel(r'/kaggle/input/house-price-new/HousePricePrediction.xlsx')

In [3]:
data.describe()

,Id,MSSubClass,LotArea,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF2,TotalBsmtSF,SalePrice
count,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2918.000000,2918.000000,1460.000000
mean,1459.000000,57.137718,10168.114080,5.564577,1971.312778,1984.264474,49.582248,1051.777587,180921.195890
std,842.787043,42.517628,7886.996359,1.113131,30.291442,20.894344,169.205611,440.766258,79442.502883
min,0.000000,20.000000,1300.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,34900.000000
25%,729.500000,20.000000,7478.000000,5.000000,1953.500000,1965.000000,0.000000,793.000000,129975.000000
50%,1459.000000,50.000000,9453.000000,5.000000,1973.000000,1993.000000,0.000000,989.500000,163000.000000
75%,2188.500000,70.000000,11570.000000,6.000000,2001.000000,2004.000000,0.000000,1302.000000,214000.000000
max,2918.000000,190.000000,215245.000000,9.000000,2010.000000,2010.000000,1526.000000,6110.000000,755000.000000


In [4]:
train_data = data.iloc[:1460, :]
test_data = data.iloc[1460:, :12]

In [5]:
train_data['Exterior1st']=train_data['Exterior1st'].fillna(train_data['Exterior1st'].mode()[0])
train_data['MSZoning']=train_data['MSZoning'].fillna(train_data['MSZoning'].mode()[0])
train_data['BsmtFinSF2']=train_data['BsmtFinSF2'].fillna(train_data['BsmtFinSF2'].mean())
train_data['TotalBsmtSF']=train_data['TotalBsmtSF'].fillna(train_data['TotalBsmtSF'].mean())

/tmp/ipykernel_35/2043804754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Exterior1st']=train_data['Exterior1st'].fillna(train_data['Exterior1st'].mode()[0])
/tmp/ipykernel_35/2043804754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['MSZoning']=train_data['MSZoning'].fillna(train_data['MSZoning'].mode()[0])
/tmp/ipykernel_35/2043804754.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [6]:
train_data

,Id,MSSubClass,MSZoning,LotArea,LotConfig,BldgType,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,BsmtFinSF2,TotalBsmtSF,SalePrice
0,0,60,RL,8450,Inside,1Fam,5,2003,2003,VinylSd,0.0,856.0,208500.0
1,1,20,RL,9600,FR2,1Fam,8,1976,1976,MetalSd,0.0,1262.0,181500.0
2,2,60,RL,11250,Inside,1Fam,5,2001,2002,VinylSd,0.0,920.0,223500.0
3,3,70,RL,9550,Corner,1Fam,5,1915,1970,Wd Sdng,0.0,756.0,140000.0
4,4,60,RL,14260,FR2,1Fam,5,2000,2000,VinylSd,0.0,1145.0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,60,RL,7917,Inside,1Fam,5,1999,2000,VinylSd,0.0,953.0,175000.0
1456,1456,20,RL,13175,Inside,1Fam,6,1978,1988,Plywood,163.0,1542.0,210000.0
1457,1457,70,RL,9042,Inside,1Fam,9,1941,2006,CemntBd,0.0,1152.0,266500.0
1458,1458,20,RL,9717,Inside,1Fam,6,1950,1996,MetalSd,1029.0,1078.0,142125.0


In [7]:
train_data.isnull().sum()

Id              0
MSSubClass      0
MSZoning        0
LotArea         0
LotConfig       0
BldgType        0
OverallCond     0
YearBuilt       0
YearRemodAdd    0
Exterior1st     0
BsmtFinSF2      0
TotalBsmtSF     0
SalePrice       0
dtype: int64

In [12]:
train_data= train_data.drop(columns=['Id'])


KeyError: "['Id'] not found in axis"

In [13]:
#Object columns: ['MSZoning', 'LotConfig', 'BldgType', 'Exterior1st']
encoding =['MSZoning', 'LotConfig', 'BldgType', 'Exterior1st']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in encoding:
    train_data[col] = le.fit_transform(train_data[col])

In [14]:
X= train_data.drop(columns=['SalePrice'])
Y= train_data['SalePrice']

In [15]:
X

,MSSubClass,MSZoning,LotArea,LotConfig,BldgType,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,BsmtFinSF2,TotalBsmtSF
0,60,3,8450,4,0,5,2003,2003,12,0.0,856.0
1,20,3,9600,2,0,8,1976,1976,8,0.0,1262.0
2,60,3,11250,4,0,5,2001,2002,12,0.0,920.0
3,70,3,9550,0,0,5,1915,1970,13,0.0,756.0
4,60,3,14260,2,0,5,2000,2000,12,0.0,1145.0
...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,7917,4,0,5,1999,2000,12,0.0,953.0
1456,20,3,13175,4,0,6,1978,1988,9,163.0,1542.0
1457,70,3,9042,4,0,9,1941,2006,5,0.0,1152.0
1458,20,3,9717,4,0,6,1950,1996,8,1029.0,1078.0


In [16]:
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN,Y_TEST = train_test_split(
    X,Y, test_size=0.2, random_state=42)

In [18]:
from sklearn.ensemble import RandomForestRegressor

model_RFR = RandomForestRegressor(n_estimators=5)
model_RFR.fit(X_TRAIN, Y_TRAIN)
Y_pred_RF = model_RFR.predict(X_TEST)

print(mean_absolute_percentage_error(Y_TEST,Y_pred_RF))
r2_scoreRF = r2_score(Y_TEST,Y_pred_RF)
print(r2_scoreRF)

0.1460136652480105
0.8079622913545726


In [21]:
import pickle

with open('model_pickle','wb') as file :
    pickle.dump(model_RFR, file)

In [22]:
with open('model_pickle', 'rb') as file :
    model_pickle= pickle.load(file)

In [24]:
testing_data= pd.read_excel("/kaggle/input/testing-data/Book.xlsx")

In [28]:
testing_data=testing_data.drop(columns=['Id'])

In [48]:
testing_data.isnull().sum()

MSSubClass      0
MSZoning        0
LotArea         0
LotConfig       0
BldgType        0
OverallCond     0
YearBuilt       0
YearRemodAdd    0
Exterior1st     0
BsmtFinSF2      1
TotalBsmtSF     1
dtype: int64

In [49]:
testing_data['BsmtFinSF2'] =  testing_data['BsmtFinSF2'].mean()
testing_data['TotalBsmtSF'] = testing_data['TotalBsmtSF'].mean()

In [53]:
testing_data.isnull().sum()

MSSubClass      0
MSZoning        0
LotArea         0
LotConfig       0
BldgType        0
OverallCond     0
YearBuilt       0
YearRemodAdd    0
Exterior1st     0
BsmtFinSF2      0
TotalBsmtSF     0
dtype: int64

In [50]:
cat_columns=[]
for i in testing_data.columns:
    if( testing_data[i].dtypes=="object"):
        cat_columns.append(i)
print(cat_columns)

[]


In [54]:
from sklearn.preprocessing import LabelEncoder
cat_columns=['MSZoning', 'LotConfig', 'BldgType', 'Exterior1st']
lable=LabelEncoder()
for i in cat_columns:
    testing_data[i]= lable.fit_transform(testing_data[i])

In [55]:
X_testing_data= testing_data

In [56]:
X_testing_data

,MSSubClass,MSZoning,LotArea,LotConfig,BldgType,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,BsmtFinSF2,TotalBsmtSF
0,20,2,11622,4,0,6,1961,1961,10,52.619342,1046.11797
1,20,3,14267,0,0,6,1958,1958,11,52.619342,1046.11797
2,60,3,13830,4,0,5,1997,1998,10,52.619342,1046.11797
3,60,3,9978,4,0,6,1998,1998,10,52.619342,1046.11797
4,120,3,5005,4,4,5,1992,1992,6,52.619342,1046.11797
...,...,...,...,...,...,...,...,...,...,...,...
1454,160,4,1936,4,3,7,1970,1970,5,52.619342,1046.11797
1455,160,4,1894,4,4,5,1970,1970,5,52.619342,1046.11797
1456,20,3,20000,4,0,7,1960,1996,10,52.619342,1046.11797
1457,85,3,10441,4,0,5,1992,1992,6,52.619342,1046.11797


In [58]:
testing_output_value = model_pickle.predict(X_testing_data)

In [61]:
testing_output_value.reshape(-1,1)

array([[137800.],
       [155660.],
       [242500.],
       ...,
       [170080.],
       [195900.],
       [222200.]])

In [62]:
new_testing_output_value= pd.DataFrame(testing_output_value)

In [63]:
new_testing_output_value

,0
0,137800.0
1,155660.0
2,242500.0
3,240400.0
4,180060.0
...,...
1454,126000.0
1455,126600.0
1456,170080.0
1457,195900.0


In [66]:
submission = pd.DataFrame([testing_data[:]], Predicted Saleprice: new_testing_output_value[0])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1721078062.py, line 1)

In [67]:
submissionDF = pd.DataFrame()

In [72]:
submissionDF = testing_data.copy()

In [73]:
submissionDF['Predicted_Sale_Price'] = testing_output_value

In [74]:
submissionDF

,MSSubClass,MSZoning,LotArea,LotConfig,BldgType,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,BsmtFinSF2,TotalBsmtSF,Predicted_Sale_Price
0,20,2,11622,4,0,6,1961,1961,10,52.619342,1046.11797,137800.0
1,20,3,14267,0,0,6,1958,1958,11,52.619342,1046.11797,155660.0
2,60,3,13830,4,0,5,1997,1998,10,52.619342,1046.11797,242500.0
3,60,3,9978,4,0,6,1998,1998,10,52.619342,1046.11797,240400.0
4,120,3,5005,4,4,5,1992,1992,6,52.619342,1046.11797,180060.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,4,1936,4,3,7,1970,1970,5,52.619342,1046.11797,126000.0
1455,160,4,1894,4,4,5,1970,1970,5,52.619342,1046.11797,126600.0
1456,20,3,20000,4,0,7,1960,1996,10,52.619342,1046.11797,170080.0
1457,85,3,10441,4,0,5,1992,1992,6,52.619342,1046.11797,195900.0


In [75]:
submissionDF.to_csv('submissionDF.csv', index=False)

In [76]:
pd.read_csv("submissionDF.csv")

,MSSubClass,MSZoning,LotArea,LotConfig,BldgType,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,BsmtFinSF2,TotalBsmtSF,Predicted_Sale_Price
0,20,2,11622,4,0,6,1961,1961,10,52.619342,1046.11797,137800.0
1,20,3,14267,0,0,6,1958,1958,11,52.619342,1046.11797,155660.0
2,60,3,13830,4,0,5,1997,1998,10,52.619342,1046.11797,242500.0
3,60,3,9978,4,0,6,1998,1998,10,52.619342,1046.11797,240400.0
4,120,3,5005,4,4,5,1992,1992,6,52.619342,1046.11797,180060.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,4,1936,4,3,7,1970,1970,5,52.619342,1046.11797,126000.0
1455,160,4,1894,4,4,5,1970,1970,5,52.619342,1046.11797,126600.0
1456,20,3,20000,4,0,7,1960,1996,10,52.619342,1046.11797,170080.0
1457,85,3,10441,4,0,5,1992,1992,6,52.619342,1046.11797,195900.0


In [78]:
submissionDF.to_csv('/kaggle/working/submissionDF.csv', index=False)
